# 1a

In [2]:
%pip install tensorflow keras scikit-learn

Note: you may need to restart the kernel to use updated packages.


c:\Users\Nooit\OneDrive\Documenten\Datascience and Artificialinteligence\MasterCS Jaar1\IDL\intro deeplearning ass1\IDL2\.venv\Scripts\python.exe: No module named pip


In [35]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.python.framework.ops import SymbolicTensor
from keras.src import ops

from get_dataset import download_data
seed=42
np.random.seed(seed)

In [59]:
url = r"https://surfdrive.surf.nl/index.php/s/Nznt5c48Mzlb2HY/download?path=%2F&files=A1_data_75.zip"
file = download_data(url)

url = r"https://surfdrive.surf.nl/index.php/s/Nznt5c48Mzlb2HY/download?path=%2F&files=A1_data_150.zip"
file = download_data(url)

X = np.load("data/A1_data_75/images.npy")
y = np.load("data/A1_data_75/labels.npy")

X = X / 255

print(X.shape)
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=seed)

print(X_train.shape, X_val.shape, X_test.shape)

Data already exists at data\A1_data_75. Skipping download.
Data already exists at data\A1_data_150. Skipping download.
(18000, 75, 75)
(18000, 2)
(10125, 75, 75) (3375, 75, 75) (4500, 75, 75)


In [60]:
batch_size = 128
num_classes = 24
epochs = 12

img_rows, img_cols = X_train.shape[1], X_train.shape[2]
input_shape = (img_rows, img_cols, 1)

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_val = X_val.reshape(X_val.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print(X_train.shape, X_val.shape, X_test.shape)

(10125, 75, 75, 1) (3375, 75, 75, 1) (4500, 75, 75, 1)


In [61]:
def to_categorical(y, num_classes):
    """
    Convert time to categorical labels.

    :param y: numpy array of shape (num_samples, 2) where the first column is hour (0-11)
              and the second column is minute (0-59)
    :param num_classes: total number of classes
    :return: numpy array with one-hot encoded labels that represent the class that the elements of y belong to
    """
    # check if it is already one-hot encoded
    if y.shape[1] == num_classes:
        return y

    class_ph = num_classes//12                    # amount of classes per hour
    class_pm = 60//class_ph                       # amount of classes per minute
    class_ = y[:, 0]*class_ph + y[:, 1]//class_pm # if num_class is 24, then per 30 min is the next class.

    return keras.utils.to_categorical(class_, num_classes)

# Convert labels to one-hot encoding
# ex.
# y_train = to_categorical(y_train, 720) # when doing 720 labels use this

y_train = to_categorical(y_train, num_classes)
print(f"Class and amount:\n{np.sum(y_train,axis=0)}\n") # print class distribution, the index is the class, the amount is how many times that class is in the data

y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

print(y_train.shape, y_val.shape, y_test.shape)

Class and amount:
[442. 431. 420. 443. 438. 403. 416. 433. 414. 435. 415. 417. 424. 409.
 430. 431. 418. 402. 410. 398. 430. 420. 412. 434.]

(10125, 24) (3375, 24) (4500, 24)


In [ ]:
def common_sense_categories_acc(y_true: SymbolicTensor, y_pred: SymbolicTensor) -> SymbolicTensor:
    """
    --------------------------------------------
    Get common sense accuracy for categories
    --------------------------------------------
    :param y_true: tensor filled with tensors with true labels
    :param y_pred: tensor filled with tensors with predicted labels
    :return: common sense accuracy score tensor

    Common sense accuracy metric formula: 
    common sense accuracy = 1-((highest possible common sence loss)/(max possible difference value)) 
    = ((max possible difference value)-(highest possible common sence loss))/(max possible difference value)
    = (amount correct)/(total possible correct amount)
    """
    # read what the class is
    y_true = tf.argmax(y_true, axis=-1) # when printed gives: tf.Tensor(0, shape=(), dtype=int64), here it gives 0 because it is class 0
    y_pred = tf.argmax(y_pred, axis=-1)

    #calc accuracy (tf.cast makes the values have dtype float32 in this case)
    diff = tf.abs(y_true - y_pred)                                # difference (not common sence yet)
    csl = tf.minimum(diff, tf.abs(diff - num_classes))            # highest possible common sence loss (=common sence difference)
    max_diff = tf.cast(tf.math.ceil(num_classes / 2), tf.float32) # max possible difference value
    acc = 1-(tf.cast(csl, tf.float32) / max_diff)                 # accuracy
    return acc


In [62]:
def common_sense_mse(y_true,y_pred):
    """
    --------------------------------------------
    Get mean squared error with common sense
    --------------------------------------------
    :param y_true: tensor filled with tensors with true labels
    :param y_pred: tensor filled with tensors with predicted labels
    :return: mean squared error (MSE) with common sense incorporated (MSE, but we make classes that are further away have more loss)

    Make a distribution that is bigger in value when farther away from the correct class. 
    That distribution is then used as weights to alter the mean squared error (MSE).
    What is returned is: MSE * distribution
    """
    # precausion, make them have the same dtype
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # get squared difference
    diff = tf.square(y_true - y_pred)  # shape (batch, num_classes)

    # get the class
    class_by_mult = tf.range(num_classes, dtype=tf.float32)
    true_class = tf.reduce_sum(y_true * class_by_mult, axis=-1) 
    true_class = tf.cast(true_class, tf.int32)  # must be int for tf.roll
    
    # get distribution based on absolute distance for when index = 0
    dist = [1]
    max_amount = np.ceil(num_classes / 2)
    for i in range(2,int(max_amount)+1):
        dist.append(i)
    for i in range(int(max_amount)+1,1,-1):
        dist.append(i)
    dist = tf.constant(dist, dtype=tf.float32)

    # rotate distribution so that 1 aligns with true class
    # Ex: when index=0 is the correct class, then the distribution 
    # for 4 classes would look like this: [1,2,3,2]
    # , when index = 1 it would look like this: [2,1,2,3]
    def rotate_dist(idx):
        idx = tf.reshape(idx, ())  # make is a scalar
        row = tf.roll(dist, shift=idx, axis=0)
        return row

    dist_matrix = tf.map_fn(rotate_dist, true_class, fn_output_signature=tf.float32) # loop over true_class rows

    # multiply squared difference by distance weights
    diff_with_common_sense = diff * dist_matrix

    # average over classes to get the MSE with common_sense incorporated
    loss = tf.reduce_mean(diff_with_common_sense, axis=-1)
    return loss


In [ ]:
# #######################################
model = Sequential([
    Input(shape=input_shape),
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])


##################
model.compile(loss=common_sense_mse,
              optimizer=keras.optimizers.Adadelta(),
              metrics=[common_sense_categories_acc])


model.summary()

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 73, 73, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 71, 71, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 35, 35, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 35, 35, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 78400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    10,035,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 24)             │         3,096 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,057,240 (38.37 MB)

 Trainable params: 10,057,240 (38.37 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 67s 814ms/step - common_sense_categories_acc: 0.4964 - loss: 0.0506 - val_common_sense_categories_acc: 0.5049 - val_loss: 0.0504
Epoch 2/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 60s 752ms/step - common_sense_categories_acc: 0.4971 - loss: 0.0506 - val_common_sense_categories_acc: 0.5052 - val_loss: 0.0504
Epoch 3/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 62s 778ms/step - common_sense_categories_acc: 0.5042 - loss: 0.0506 - val_common_sense_categories_acc: 0.5053 - val_loss: 0.0504
Epoch 4/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 57s 717ms/step - common_sense_categories_acc: 0.4991 - loss: 0.0506 - val_common_sense_categories_acc: 0.5049 - val_loss: 0.0504
Epoch 5/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 49s 613ms/step - common_sense_categories_acc: 0.4944 - loss: 0.0506 - val_common_sense_categories_acc: 0.5055 - val_loss: 0.0504
Epoch 6/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 66s 824ms/step - common_sense_categories_acc: 0.5007 - loss: 0.0506 - val_common_sense_categories_acc: 0.5066 - val_loss: 0.050

In [10]:
model = Sequential([
    Input(shape=input_shape),
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

model.compile(loss=keras.losses.MSE,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])



model.summary()

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 73, 73, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 71, 71, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 35, 35, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 35, 35, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 78400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    10,035,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 24)             │         3,096 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,057,240 (38.37 MB)

 Trainable params: 10,057,240 (38.37 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 60s 733ms/step - accuracy: 0.0418 - loss: 0.0400 - val_accuracy: 0.0391 - val_loss: 0.0399
Epoch 2/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 49s 607ms/step - accuracy: 0.0439 - loss: 0.0400 - val_accuracy: 0.0391 - val_loss: 0.0399
Epoch 3/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 52s 647ms/step - accuracy: 0.0417 - loss: 0.0400 - val_accuracy: 0.0394 - val_loss: 0.0399
Epoch 4/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 55s 691ms/step - accuracy: 0.0403 - loss: 0.0400 - val_accuracy: 0.0391 - val_loss: 0.0399
Epoch 5/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 41s 507ms/step - accuracy: 0.0411 - loss: 0.0400 - val_accuracy: 0.0388 - val_loss: 0.0399
Epoch 6/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 41s 512ms/step - accuracy: 0.0385 - loss: 0.0400 - val_accuracy: 0.0385 - val_loss: 0.0399
Epoch 7/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 38s 479ms/step - accuracy: 0.0398 - loss: 0.0400 - val_accuracy: 0.0379 - val_loss: 0.0399
Epoch 8/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 43s 543ms/step - accuracy: 0.0405 - loss: 0.0400 - val_accu